In [1]:
import pandas as pd
import pandas_ta as pta
import numpy as np
from talib import abstract as ta

path = '../data/TSLA_10y.csv'
symbol = path.split('/')[-1].split('_')[0].replace('USDT', '')
df = pd.read_csv(path)
df.head()

,time,open,high,low,close,volume,Dividends,Stock Splits
0,2014-02-24 05:00:00+00:00,13.917333,14.557333,13.888000,14.510000,124176000,0.0,0.0
1,2014-02-25 05:00:00+00:00,15.333333,17.280001,15.230000,16.533333,490225500,0.0,0.0
2,2014-02-26 05:00:00+00:00,17.238667,17.666668,16.500000,16.866667,369069000,0.0,0.0
3,2014-02-27 05:00:00+00:00,17.416668,17.459999,16.555332,16.836000,269187000,0.0,0.0
4,2014-02-28 05:00:00+00:00,16.643333,16.845333,16.170000,16.320667,218847000,0.0,0.0


In [6]:
ta.CDL2CROWS(df['open'], df['high'], df['low'], df['close'])

array([0, 0, 0, ..., 0, 0, 0])

In [3]:
temp = df.ta.bbands()
df['BBANDS_U'] = temp['BBU_5_2.0']
df['BBANDS_L'] = temp['BBL_5_2.0']
df['BBANDS_M'] = temp['BBM_5_2.0']
df['DC_L'] = df.ta.donchian()['DCL_20_20']
df['DC_U'] = df.ta.donchian()['DCU_20_20']
df['SMA_20'] = df.ta.sma(length=20)
df['SMA_50'] = df.ta.sma(length=50)
df['SMA_200'] = df.ta.sma(length=200)
temp = df.ta.cdl_pattern(name="closingmarubozu")
df['MARUBOZU'] = temp['CDL_CLOSINGMARUBOZU'].replace(100,1).replace(-100,-1)
#singal_buy = MARUBOZU == 100 and close > DONCH_U 
#singal_sell = MARUBOZU == -100 and close < DONCH_L
df['Price_above_SMA_20'] = np.where(df['close'] > df['SMA_20'], 1, 0)
df['Price_above_SMA_50'] = np.where(df['close'] > df['SMA_50'], 1, 0)
df['Price_above_SMA_200'] = np.where(df['close'] > df['SMA_200'], 1, 0)
df['Signal_Buy'] = np.where((df['close'] > df['BBANDS_L']) & (df['close'] < df['BBANDS_M']), 1, 0)
df['Signal_Sell'] = np.where((df['close'] < df['BBANDS_U']) & (df['close'] > df['BBANDS_M']), 1, 0)
df['Signal'] = df['Signal_Buy'].replace(1,2) + df['Signal_Sell'].replace(1,3)
df['ATR'] = df.ta.atr()
df['ADX'] = df.ta.adx()['ADX_14']
#help(ta.adx)

In [4]:
df['Signal'].value_counts() , df['Signal_Buy'].value_counts(), df['Signal_Sell'].value_counts()

(Signal
 3    1342
 2    1170
 0       4
 Name: count, dtype: int64,
 Signal_Buy
 0    1346
 1    1170
 Name: count, dtype: int64,
 Signal_Sell
 1    1342
 0    1174
 Name: count, dtype: int64)

In [5]:
help(ta.donchian)

Help on function donchian in module pandas_ta.volatility.donchian:

donchian(high, low, lower_length=None, upper_length=None, offset=None, **kwargs)
    Donchian Channels (DC)
    
    Donchian Channels are used to measure volatility, similar to
    Bollinger Bands and Keltner Channels.
    
    Sources:
        https://www.tradingview.com/wiki/Donchian_Channels_(DC)
    
    Calculation:
        Default Inputs:
            lower_length=upper_length=20
        LOWER = low.rolling(lower_length).min()
        UPPER = high.rolling(upper_length).max()
        MID = 0.5 * (LOWER + UPPER)
    
    Args:
        high (pd.Series): Series of 'high's
        low (pd.Series): Series of 'low's
        lower_length (int): The short period. Default: 20
        upper_length (int): The short period. Default: 20
        offset (int): How many periods to offset the result. Default: 0
    
    Kwargs:
        fillna (value, optional): pd.DataFrame.fillna(value)
        fill_method (value, optional): Type

In [6]:
#dropna
df = df.dropna()
df.head()

,time,open,high,low,close,volume,Dividends,Stock Splits,BBANDS_U,BBANDS_L,...,SMA_200,MARUBOZU,Price_above_SMA_20,Price_above_SMA_50,Price_above_SMA_200,Signal_Buy,Signal_Sell,Signal,ATR,ADX
199,2014-12-05 05:00:00+00:00,15.244667,15.292667,14.817333,14.914000,90954000,0.0,0.0,15.641731,14.874536,...,15.489737,0.0,0,0,0,1,0,2,0.507733,22.029153
200,2014-12-08 05:00:00+00:00,14.769333,14.990667,14.156000,14.290667,138384000,0.0,0.0,15.837895,14.217572,...,15.488640,0.0,0,0,0,1,0,2,0.531085,24.311505
201,2014-12-09 05:00:00+00:00,13.956000,14.515333,13.618000,14.459333,141472500,0.0,0.0,15.631294,14.036440,...,15.478270,0.0,0,0,0,1,0,2,0.557246,26.868579
202,2014-12-10 05:00:00+00:00,14.275333,14.451333,13.846667,13.989333,109711500,0.0,0.0,15.453698,13.695102,...,15.463883,0.0,0,0,0,1,0,2,0.561204,29.243005
203,2014-12-11 05:00:00+00:00,14.035333,14.362000,13.882000,13.925333,100416000,0.0,0.0,15.030224,13.601242,...,15.449330,0.0,0,0,0,1,0,2,0.555404,31.447829


In [7]:
#set time to index

df['time'] = pd.to_datetime(df['time'])
df = df.set_index('time')
df.head()

C:\Users\hiban\AppData\Local\Temp\ipykernel_27424\1852176384.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time'] = pd.to_datetime(df['time'])


,open,high,low,close,volume,Dividends,Stock Splits,BBANDS_U,BBANDS_L,BBANDS_M,...,SMA_200,MARUBOZU,Price_above_SMA_20,Price_above_SMA_50,Price_above_SMA_200,Signal_Buy,Signal_Sell,Signal,ATR,ADX
time,,,,,,,,,,,,,,,,,,,,,
2014-12-05 05:00:00+00:00,15.244667,15.292667,14.817333,14.914000,90954000,0.0,0.0,15.641731,14.874536,15.258134,...,15.489737,0.0,0,0,0,1,0,2,0.507733,22.029153
2014-12-08 05:00:00+00:00,14.769333,14.990667,14.156000,14.290667,138384000,0.0,0.0,15.837895,14.217572,15.027733,...,15.488640,0.0,0,0,0,1,0,2,0.531085,24.311505
2014-12-09 05:00:00+00:00,13.956000,14.515333,13.618000,14.459333,141472500,0.0,0.0,15.631294,14.036440,14.833867,...,15.478270,0.0,0,0,0,1,0,2,0.557246,26.868579
2014-12-10 05:00:00+00:00,14.275333,14.451333,13.846667,13.989333,109711500,0.0,0.0,15.453698,13.695102,14.574400,...,15.463883,0.0,0,0,0,1,0,2,0.561204,29.243005
2014-12-11 05:00:00+00:00,14.035333,14.362000,13.882000,13.925333,100416000,0.0,0.0,15.030224,13.601242,14.315733,...,15.449330,0.0,0,0,0,1,0,2,0.555404,31.447829


In [8]:
#rename columns
df.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)

In [9]:
df['Signal'].value_counts()

Signal
3    1235
2    1082
Name: count, dtype: int64

In [10]:
from backtesting import Strategy
from backtesting import Backtest

def SIGNAL(df,col_name):
    return df[col_name]

class Weight_Strategy(Strategy):
    atr_sl_rate = 1.3
    TPSL_rate = 1.5
    save_size_rate = 0.125
    init_cash = 10000
    profit_target = 12000
    size_multiplier_rate = 0.5
    
    # long params
    lp1 = 1
    lp2 = 1
    lp3 = 1
    lv1 = 3

    def init(self):
        super().init()
        self.signal = self.I(SIGNAL, self.data, 'Signal')

    def next(self):
        super().next()

        slatr = self.atr_sl_rate * self.data.ATR[-1]
        size_multiplier = self.size_multiplier_rate if self.equity >= self.profit_target else 1.0
        buy_confidence = (self.lp1 * self.data.MARUBOZU[-1] + self.lp2 * self.data.Price_above_SMA_20[-1] + self.lp3 * self.data.Price_above_SMA_50[-1])

        if self.signal == 2 and buy_confidence >= self.lv1:
            sl_price = self.data.Close[-1] - slatr
            tp_price = self.data.Close[-1] + self.TPSL_rate * slatr
            size = min(self.save_size_rate * self.equity / slatr, self.init_cash / slatr) * size_multiplier
            if self.equity > self.init_cash * 1.5:
                self.buy(sl=sl_price, tp=tp_price, size=int(size))
            else:
                self.buy(sl=sl_price, tp=tp_price)
            
cash = 10000
bt = Backtest(df, Weight_Strategy, cash=cash, commission=.00002, margin=.05)
stat = bt.run()
stat

Start                     2014-12-05 05:00...
End                       2024-02-21 05:00...
Duration                   3365 days 00:00:00
Exposure Time [%]                    3.366422
Equity Final [$]                  29526.47905
Equity Peak [$]                  29977.330744
Return [%]                          195.26479
Buy & Hold Return [%]             1205.954204
Return (Ann.) [%]                   12.496975
Volatility (Ann.) [%]               32.320606
Sharpe Ratio                         0.386657
Sortino Ratio                        1.810963
Calmar Ratio                         0.853408
Max. Drawdown [%]                   -14.64362
Avg. Drawdown [%]                   -6.171938
Max. Drawdown Duration     1114 days 00:00:00
Avg. Drawdown Duration      222 days 00:00:00
# Trades                                   10
Win Rate [%]                             70.0
Best Trade [%]                      17.533384
Worst Trade [%]                     -6.661959
Avg. Trade [%]                    

In [11]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Scatter(x=stat['_equity_curve'].index, y=stat['_equity_curve']['Equity'])])
fig.show()

In [12]:
%%time
stats = bt.optimize(atr_sl_rate = np.arange(0.2, 1.6, 0.2).tolist(),
                    TPSL_rate = np.arange(0.2, 1.6, 0.2).tolist(),
                    lp1 = np.arange(0.2, 1.2, 0.2).tolist(),
                    lp2 = np.arange(0.2, 1.2, 0.2).tolist(),
                    lp3 = np.arange(0.2, 1.2, 0.2).tolist(),
                    lv1 = np.arange(0.2, 1.2, 0.2).tolist(),
                    maximize='SQN')
stats

C:\Users\hiban\AppData\Roaming\Python\Python311\site-packages\backtesting\backtesting.py:1488: UserWarning:

Searching for best of 30625 configurations.



  0%|          | 0/103 [00:00<?, ?it/s]

CPU times: total: 27min 24s
Wall time: 27min 35s


Start                     2014-12-05 05:00...
End                       2024-02-21 05:00...
Duration                   3365 days 00:00:00
Exposure Time [%]                   17.220544
Equity Final [$]                146332.352205
Equity Peak [$]                 153184.160201
Return [%]                        1363.323522
Buy & Hold Return [%]             1205.954204
Return (Ann.) [%]                   33.888713
Volatility (Ann.) [%]               90.631564
Sharpe Ratio                         0.373917
Sortino Ratio                        1.117973
Calmar Ratio                         0.453426
Max. Drawdown [%]                  -74.739294
Avg. Drawdown [%]                   -7.774085
Max. Drawdown Duration      805 days 00:00:00
Avg. Drawdown Duration       75 days 00:00:00
# Trades                                  316
Win Rate [%]                        62.341772
Best Trade [%]                      17.235369
Worst Trade [%]                     -7.440963
Avg. Trade [%]                    

In [13]:
stats.keys()

Index(['Start', 'End', 'Duration', 'Exposure Time [%]', 'Equity Final [$]',
       'Equity Peak [$]', 'Return [%]', 'Buy & Hold Return [%]',
       'Return (Ann.) [%]', 'Volatility (Ann.) [%]', 'Sharpe Ratio',
       'Sortino Ratio', 'Calmar Ratio', 'Max. Drawdown [%]',
       'Avg. Drawdown [%]', 'Max. Drawdown Duration', 'Avg. Drawdown Duration',
       '# Trades', 'Win Rate [%]', 'Best Trade [%]', 'Worst Trade [%]',
       'Avg. Trade [%]', 'Max. Trade Duration', 'Avg. Trade Duration',
       'Profit Factor', 'Expectancy [%]', 'SQN', '_strategy', '_equity_curve',
       '_trades'],
      dtype='object')

In [14]:
stats._strategy

<Strategy Weight_Strategy(atr_sl_rate=1.0,TPSL_rate=0.4,lp1=0.4,lp2=0.4,lp3=0.2,lv1=0.4)>

In [15]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Scatter(x=stats['_equity_curve'].index, y=stats['_equity_curve']['Equity'])])
fig.update_layout(title='Profit | '+symbol+' | '+ str(cash), yaxis_title='USDT', xaxis_title='Date')
fig.show()